In [1]:
# SciPy imports
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
import seaborn as sns
import pickle
import datetime
import random

# Other necessary imports
import sqlite3
import ast
from collections import defaultdict
import pprint

In [22]:
conn = sqlite3.connect('database.sqlite')
cursor = conn.cursor()
print("Opened database successfully")

Opened database successfully


# Strategy

- subset to last 10 days of searches (timeframe for testSearchStream + 1 day)
- select last 7 searches per User for val/test
- historical searches are everything prior

**Note:** any query beginning with "CREATE TABLE" was compiled in batch_create_tables.py

*See README.md for more information*

### Restrict Timeframe

In [3]:
## creating a table with the last 10 days of searches
q = '''
CREATE TABLE last10days AS
    SELECT * FROM SearchInfo2
    WHERE SearchDate > "2015-05-11 00:00:00.0"
    ORDER BY SearchDate DESC;
'''
%time cursor.executescript(q)

CPU times: user 8min 34s, sys: 5min 46s, total: 14min 21s
Wall time: 32min 45s


In [ ]:
## take the search IDs from above query and grab the related impressions
q ='''
CREATE TABLE last10days_imp AS
    SELECT * FROM trainSearchStream
    WHERE SearchID IN (
    SELECT SearchID FROM last10days 
    ORDER BY SearchID ASC);
'''
%time cursor.executescript(q)

#### Sanity Check!

In [5]:
# compare count of SearchIDs from last10days (of SearchInfo)
# to count of SearchIDs when the correlated impressions are queried
q1 = '''
SELECT COUNT(DISTINCT(SearchID))
FROM last10days'''
q2 = '''
SELECT COUNT(DISTINCT(SearchID))
FROM last10days_imp'''
print('Distinct SearchIDs from last10days: ', cursor.execute(q1).fetchone()[0])
print('Distinct SearchIDs from last10days of impressions: ', cursor.execute(q2).fetchone()[0])

Distinct SearchIDs from last9days:  39009184
Distinct SearchIDs from last9days of impressions:  39009184


In [ ]:
# gets correlated Ad information for ads in last10days_imp
q ='''
CREATE TABLE last10days_ads AS
    SELECT * FROM AdsInfo
    WHERE AdID IN (
    SELECT AdID FROM last10days_imp 
    ORDER BY AdID ASC);
'''
%time cursor.executescript(q)

### MERGE

In [4]:
# merge all those tables together for convenience
q ='''
CREATE TABLE last10days_merged AS

    SELECT i.*, a.*, s.*
    FROM last10days_imp i
    JOIN (
          SELECT * FROM last10days 
         ) s
    ON s.SearchID = i.SearchID
    JOIN last10days_ads a
    ON a.AdID = i.AdID;
'''
%time cursor.executescript(q)

CPU times: user 19min 14s, sys: 1h 1min 23s, total: 1h 20min 37s
Wall time: 1h 46min 20s


In [14]:
## get history of landing page visits for Users in last9days
q ='''
CREATE TABLE hist_visits AS
    SELECT * FROM VisitsStream
    WHERE ViewDate > "2015-05-11 00:00:00.0"
    AND   UserID IN (
            SELECT UserID FROM last10days);
'''
%time cursor.executescript(q)

CPU times: user 4min 57s, sys: 5min 59s, total: 10min 56s
Wall time: 17min 44s


# Separating Validation & Test Sets

In [17]:
## for the last 7 days of Searches,
## rank Searches per User in reverse order
last_seven_searches = '''
CREATE TABLE last_seven_searches AS

SELECT last_seven.*
FROM
    (SELECT UserID, SearchID, SearchDate,
         row_number() OVER (PARTITION BY UserID ORDER BY SearchDate DESC) AS row_n
     FROM last10days) last_seven
WHERE row_n <= 7
;
'''
%time cursor.execute(last_seven_searches)

CPU times: user 1min 38s, sys: 1min 11s, total: 2min 50s
Wall time: 5min 47s


In [18]:
# for the last 7 days of Searches,
## rank Searches per User in reverse order
# take all searches after 7th-to-last and lump together as "historical"
historical_searches = '''
CREATE TABLE hist_searches AS

SELECT last_seven.*
FROM
    (SELECT UserID, SearchID, SearchDate,
         row_number() OVER (PARTITION BY UserID ORDER BY SearchDate DESC) AS row_n
     FROM last10days) last_seven
WHERE row_n > 7
;
'''
%time cursor.execute(historical_searches)

CPU times: user 1min 37s, sys: 54.1 s, total: 2min 31s
Wall time: 3min 42s


#### Quick Count

In [19]:
# print count of UserIDs in each round of searches for Last Searches #1-7 (in reverse)
for n in range(7,0,-1):
    q = f'''
    SELECT COUNT(DISTINCT(UserID)) FROM last_seven_searches
    WHERE row_n = ?;
    '''
    print(f'Number of UserIds for search# {n}: ', cursor.execute(q, (n,)).fetchone()[0])

Number of UserIds for search# 7:  1091581
Number of UserIds for search# 6:  1209443
Number of UserIds for search# 5:  1356606
Number of UserIds for search# 4:  1542396
Number of UserIds for search# 3:  1790899
Number of UserIds for search# 2:  2156078
Number of UserIds for search# 1:  2756369


> Every User has at least 1 search, with those having more-than-one decreasing in quantity.

### Build the Train/Test Sets

In [28]:
# building Training/Validation Set
train = '''
SELECT 
    m.SearchID, AdID, m.UserID, m.SearchDate,
    Price, Position, LENGTH(Title) title_length, 
    IsClick, IsContext, IsUserLoggedOn,
    CategoryID CategoryID_s,
    "CategoryID:1" CategoryID_a,
    Params, SearchParams, SearchQuery,
    tr.row_n
FROM last10days_merged m

JOIN last_seven_searches tr
ON m.SearchID = tr.SearchID
-- only take impressions from the seventh-to-last to second-to-last search per User
WHERE IsContext = 1 AND
      tr.row_n > 1  AND
      m.SearchDate >= "2015-05-12 00:00:00.0"
'''

In [33]:
# building holdout Test Set
test = '''
SELECT 
    m.SearchID, AdID, m.UserID,  m.SearchDate,
    Price, Position, LENGTH(Title) title_length, 
    IsClick, IsContext, IsUserLoggedOn,
    CategoryID CategoryID_s,
    "CategoryID:1" CategoryID_a,
    Params, SearchParams, SearchQuery,
    tr.row_n
FROM last10days_merged m

JOIN last_seven_searches tr
ON m.SearchID = tr.SearchID

-- only take impressions from the last search per User
WHERE IsContext = 1 AND
      tr.row_n = 1 AND
      m.SearchDate >= "2015-05-12 00:00:00.0"
'''

In [29]:
%time train_raw = pd.read_sql_query(train, conn)

CPU times: user 6min 40s, sys: 4min 44s, total: 11min 24s
Wall time: 16min 14s


In [34]:
%time test_raw = pd.read_sql_query(test, conn)

CPU times: user 2min 30s, sys: 1min 59s, total: 4min 29s
Wall time: 7min 44s


In [35]:
# pickle it!
train_raw.to_pickle(open('pickle/train_raw.pkl.gzip', 'wb'), compression='gzip')
test_raw.to_pickle(open('pickle/test_raw.pkl.gzip', 'wb'), compression='gzip')

# Train/Val Split

In [36]:
def train_val_split(df, components, col_name):
    '''
    Builds a dictionary of DataFrame indices split into a Walk-Forward Validation
    scheme. Each "step" or split comprises of an initial "train_set" for model fitting, 
    a "val_set" for model evaluation, and an "agg_set_v" which is the historical data that
    will be merged onto the "val_set".  After the first split, the remaining splits will 
    also have a "agg_set_t" - the historical data that will be merged into the training set.
    
    For example, with 2 splits of search items [3rd-last, 2nd-last, last]:
    ROUND 1: 
    Model is Fit on 'Train Set 1' (3rd-last) 
    Model is Scored on 'Val Set 1' (2nd-last) 
                        w/Agg Set V1 (3rd-last) as supporting historical data
    
    ROUND 2:
    Model is Fit on 'Train Set 2' (2nd-last)
                        w/ Agg Set T2 (3rd-last) as supporting historical data
    Model is Scored on 'Val Set 2' (last)
                        w/ Agg Set V2 (2nd-last) as supporting historical data
    
    Parameters
    ----------
    df : DataFrame
        Data to be split.
    
    components : list of integers
        Sequence of time-series events, listed as reverse-ordinal events.
            e.g. [3, 2, 1] for third-to-last, second-to-last, last
        
    col_name : string
        Column name that references the time-series event
    
    Returns
    -------
    train_val_splits 
        Dictionary of indices corresponding to splits.
        Number of splits is 1 - len(components).
    '''
    # instantiate defaultdict to contain splits
    train_val_splits = defaultdict()
    # loop over number of splits (number of components - 1)
    for i in range(len(components)-1):
        # for first split, train set & agg_set for validation will be the same
        # no explicit agg_set for training 
        if i == 0:
            train_val_splits[f'train_set_{i+1}'] = df[df[col_name] == components[i]].index.values
            train_val_splits[f'agg_set_v{i+1}'] = df[df[col_name] > components[i+1]].index.values
            train_val_splits[f'val_set_{i+1}'] = df[df[col_name] == components[i+1]].index.values
        else:
            # agg_set_t takes all indices prior to train_set indices
            train_val_splits[f'agg_set_t{i+1}'] = df[df[col_name] > components[i]].index.values
            # train_set takes all indices at component
            train_val_splits[f'train_set_{i+1}'] = df[df[col_name] == components[i]].index.values
            # agg_set_v takes all indices prior to val_set indices
            train_val_splits[f'agg_set_v{i+1}'] = df[df[col_name] > components[i+1]].index.values
            # val_set takes all indices at subsequent component
            train_val_splits[f'val_set_{i+1}'] = df[df[col_name] == components[i+1]].index.values
            
    return train_val_splits

In [37]:
# get index splits for training/validation
train_val_dict = train_val_split(train_raw, components=[7,6,5,4,3,2], col_name='row_n')
# pickle it
pickle.dump(train_val_dict, open('pickle/train_val_dict.pkl', 'wb'))

#### Sanity Check!

In [38]:
## statements should all equal 'True'
check1 = np.array_equal(train_val_dict['train_set_1'], train_val_dict['agg_set_v1'])
print('First split train_set and agg_set_v1 equivalent: ', check1)

check2 = np.array_equal(train_val_dict['val_set_2'], train_val_dict['train_set_3'])
print('Training Set 3 is the same as Validation Set 2: ', check2)

check3 = np.array_equal(train_val_dict['agg_set_t4'], train_val_dict['agg_set_v3'])
print('Agg Set for Training 4 is the same as Agg Set for Validation 3: ', check3)

First split train_set and agg_set_v1 equivalent:  True
Training Set 3 is the same as Validation Set 2:  True
Agg Set for Training 4 is the same as Agg Set for Validation 3:  True


# Baseline Modeling

In [183]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

In [149]:
# cast Price column as numeric and replace empty values with NaNs
def make_price_numeric(df):
    df['Price'] = df['Price'].replace('', np.NaN)
    return df.astype({'Price': 'float'})

In [241]:
train_baseline = make_price_numeric(train_raw)

In [81]:
train_baseline.dtypes

SearchID            int64
AdID                int64
UserID              int64
SearchDate         object
Price             float64
Position            int64
title_length        int64
IsClick             int64
IsContext           int64
IsUserLoggedOn      int64
CategoryID_s       object
CategoryID_a        int64
Params             object
SearchParams       object
SearchQuery        object
row_n               int64
dtype: object

In [120]:
# basic features and target
features = ['Price', 'Position', 'title_length', 'CategoryID_a']
target = ['IsClick']

In [242]:
# subset training/validation DataFrame to only contain features/target columns
train_baseline = train_baseline[features + target]
# some null prices need to have an imputed value - using 0 for simplicity
train_baseline['Price'] = train_baseline['Price'].fillna(0)

In [247]:
# set empty array for auc_scores - we will later take the mean of all 5 rounds of validation
auc_scores = np.zeros(5)

for i in range(1, 6):
    
    # get indices for splits
    t_index = train_val_dict[f'train_set_{i}']
    v_index = train_val_dict[f'val_set_{i}']
    
    # filter full train_df for splits
    t = train_baseline.iloc[t_index]
    v = train_baseline.iloc[v_index]
    
    # scale/encode features
    preprocess = make_column_transformer(
        (['Price', 'title_length'], StandardScaler()),
        (['Position', 'CategoryID_a'], OneHotEncoder(categories='auto', 
                                                     handle_unknown='ignore'))
                                        )

    # split train into features(X) and target(y)
    X_t = t.drop(columns=['IsClick'])
    X_ts = preprocess.fit_transform(X_t)
    y_t = t.IsClick
    
    # split val into features(X) and target(y)
    X_v = v.drop(columns=['IsClick'])
    X_vs = preprocess.transform(X_v)
    y_v = v.IsClick
    
    # log round # and train/val dataframe shapes
    print(f'Round {i}')
    print('X_train.shape: ', X_t.shape)
    print('X_val.shape: ', X_v.shape)
    
    # instantiate baseline model
    m = LogisticRegression(solver='liblinear')
    # fit on scaled/encoded training set
    m.fit(X_ts, y_t)
    # predict probability of target class for scaled/encoded validation set
    preds = m.predict_proba(X_vs)[:,1]
    # get AUC score
    auc = roc_auc_score(y_v, preds)
    # insert AUC score in numpy array
    auc_scores[i-1] = auc
    # print AUC score for current round
    print(f'Baseline AUC = {auc}', '\n')

# print final mean AUC for all validation rounds
print('*' * 15)
print(f'Mean AUC over 5-rounds: {np.mean(auc_scores)}')

Round 1
X_train.shape:  (1526110, 4)
X_val.shape:  (1707463, 4)
Baseline AUC = 0.6477311237604879 

Round 2
X_train.shape:  (1707463, 4)
X_val.shape:  (1934930, 4)
Baseline AUC = 0.6502892716940246 

Round 3
X_train.shape:  (1934930, 4)
X_val.shape:  (2226560, 4)
Baseline AUC = 0.657265183533147 

Round 4
X_train.shape:  (2226560, 4)
X_val.shape:  (2619364, 4)
Baseline AUC = 0.666012287642193 

Round 5
X_train.shape:  (2619364, 4)
X_val.shape:  (3196045, 4)
Baseline AUC = 0.6757942967677281 

***************
Mean AUC over 5-rounds: 0.659418432679516


## If you wanted to test baseline on Kaggle:

#### Create Train Set

In [235]:
# combine all last 7 searches for each User
tr = pd.concat([train_raw, test_raw])
# convert Price column to numeric
tr = make_price_numeric(tr)
# subset DataFrame to only include 'features' and 'target' columns
tr = tr[features + target]
# replace NaN prices with 0
tr['Price'] = tr['Price'].fillna(0)

#### Create Test Set

In [211]:
# pull in kaggle test set from sql
q = '''
SELECT TestID, AdID, Price, Position, 
       LENGTH(Title) title_length, 
       "CategoryID:1" CategoryID_a
FROM test_merged
'''
te = pd.read_sql_query(q, conn)
# convert Price column to numeric
te = make_price_numeric(te)
# replace NaN prices with 0
te['Price'] = te['Price'].fillna(0)
# some categories in this column were Null so we have to add this arbitrary value
te['CategoryID_a'] = te['CategoryID_a'].replace('', '-1')

#### Preprocess, Model, Predict

In [237]:
# scale encode/columns like before
preprocess = make_column_transformer(
    (['Price', 'title_length'], StandardScaler()),
    (['Position', 'CategoryID_a'], OneHotEncoder(categories='auto', handle_unknown='ignore'))
)

# split train into features(X) and target(y)
X_t = tr.drop(columns=['IsClick'])
X_ts = preprocess.fit_transform(X_t)
y_t = tr.IsClick

# split test into features(X) NOTE: kaggle test_set has no target included - that's what we're predicting
X_te = te
X_tes = preprocess.transform(X_te)

# train model and predict probability of IsClick for test
m = LogisticRegression(solver='liblinear')
m.fit(X_ts, y_t)
preds = m.predict_proba(X_tes)[:,1]

In [238]:
# set predictions to 'IsClick' column
X_te['IsClick'] = preds
# write to csv - this is the correct format for submission
X_te[['TestId', 'IsClick']].to_csv(open('submission.csv', 'w'), header=['ID', 'IsClick'], index=False)

**Note: Keep in mind that we've been using AUC to evaluate rather than the competition metric - LogLoss.**

That said, the above gets you just under 50th percentile or so submissions on the Private Leaderboard with a LogLoss ~0.04982.  You'll see all the other Logistic Regression scripts around there.